# Лабораторная работа №3
## Выполнил студент группы БСТ2002 Максимов И.А.

## Задание №1
### Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

### Ввод с клавиатуры

In [1]:
import timeit

main_string = 'замок закрыт на большущий ЗАМОК'
string_to_find = 'ЗАМОК'
is_case_sensitive = False

print('Введите основную строку или оставьте пустой для строки по умолчанию: ')
buffer = input()
if buffer != '':
    main_string = buffer
    buffer = ''
else:
    print('По умолчанию: замок закрыт на большущий ЗАМОК')

print('Введите строку для поиска или оставьте пустой для строки по умолчанию: ')
buffer = input()
if buffer != '':
    string_to_find = buffer
    buffer = ''
else:
    print('По умолчанию: ЗАМОК')

print('Должен ли поиск быть чувствительным к регистру?')
buffer = input()
if (buffer == 'Да') or (buffer == 'да'):
    is_case_sensitive = True

Введите основную строку или оставьте пустой для строки по умолчанию: 


По умолчанию: замок закрыт на большущий ЗАМОК
Введите строку для поиска или оставьте пустой для строки по умолчанию: 


По умолчанию: ЗАМОК
Должен ли поиск быть чувствительным к регистру?


 да


In [2]:
def prefix(s):
    str_prefix = [0]*len(s)
    for i in range(1, len(s)):
        k = str_prefix[i-1]
        while k > 0 and s[k] != s[i]:
            k = str_prefix[k-1]
        if s[k] == s[i]:
            k = k + 1
        str_prefix[i] = k
    return str_prefix


def kmp(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    index = -1
    str_prefix = prefix(s_main)
    k = 0
    for i in range(len(s_sought)):
        while k > 0 and s_main[k] != s_sought[i]:
            k = str_prefix[k-1]
        if s_main[k] == s_sought[i]:
            k = k + 1
        if k == len(s_main):
            index = i - len(s_main) + 1
            break
    return index

## Пример работы

In [3]:
print(kmp(string_to_find, main_string, is_case_sensitive))
print(kmp(string_to_find, main_string, False))

26
0


## Упрощённый алгоритм Бойера-Мура

In [4]:
def build_table(s):
    dic_table = dict()
    length = len(s)
    for i in range(length - 1):
        dic_table[s[i]] = length - i - 1
    if not s[length - 1] in dic_table:
        dic_table[s[length - 1]] = length
    return dic_table


def boyer_mur_search(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    dic_table = build_table(s_sought)
    s_sought_len = len(s_sought)
    current_step = s_sought_len - 1

    while current_step < len(s_main):  # основной цикл для поиска подстроки

        s_sought_i = s_sought_len - 1  # индекс для подстроки
        for i in range(current_step, current_step - s_sought_len, -1):
            if s_main[i] != s_sought[s_sought_i]:  # слово не совпало. Указываем необходимый шаг и выходим из for
                if s_main[i] in dic_table:
                    step = dic_table[s_main[i]]
                else:
                    step = s_sought_len
                break

            if i == current_step - s_sought_len + 1:  # последняя итерация for. Все символы совпали, возвращаем индекс
                return i
            s_sought_i -= 1

        current_step += step  # двигаем место поиска на шаг

## Пример работы

In [5]:
print(boyer_mur_search(main_string, string_to_find, case_sensitive=is_case_sensitive))
print(boyer_mur_search(main_string, string_to_find, case_sensitive=False))

26
0


## Сравнение скорости работы

In [6]:
import timeit

print("Поиск Бойера-Мура          " + str(timeit.timeit("boyer_mur_search(main_string, string_to_find, case_sensitive=is_case_sensitive)", number=1, globals=globals())))
print("Поиск Кнута-Морриса-Пратта " + str(timeit.timeit("kmp(string_to_find, main_string, is_case_sensitive)", number=1, globals=globals())))
print("Поиск по умолчанию         " + str(timeit.timeit("main_string.find(string_to_find)", number=1, globals=globals())))

Поиск Бойера-Мура          1.4900000678608194e-05
Поиск Кнута-Морриса-Пратта 1.550000160932541e-05
Поиск по умолчанию         1.2999989849049598e-06


# Задание №2

## Написать программу, определяющую, является ли данное расположение «решаемым», то есть можно ли из него за конечное число шагов перейти к правильному. Если это возможно, то необходимо найти хотя бы одно решение - последовательность движений, после которой числа будут расположены в правильном порядке. Входные данные: массив чисел, представляющий собой расстановку в порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле. Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет собой «решенную» позицию элементов.